In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import random\n",
    "import hashlib\n",
    "import json\n",
    "from faker import Faker\n",
    "fake = Faker('ru_RU')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_group(n=5):\n",
    "    group = ('ИУ','АК','МТ','РК','ФМ')\n",
    "    return random.choice(group) + str(random.randint(1, n))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_teacher_position():\n",
    "    positions = ('профессор','доцент','старший преподаватель','преподаватель','ассистент','заведующий кафедрой')\n",
    "    return random.choices(positions, (8, 20, 20, 20, 10, 1), k=1)[0]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_course_name(n=1):\n",
    "    with open('subject.json','r') as f:\n",
    "        subjects = json.load(f)\n",
    "    return random.choice(subjects) + ' ' + str(random.randint(1, n))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_name_gender():\n",
    "    gender = random.choice(('M', 'F'))\n",
    "    name = fake.name_male() if gender == 'M' else fake.name_female()\n",
    "    name = name.split()\n",
    "    if len(name) > 3: name = name[1:4]\n",
    "    return name + [gender]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_birthday_role(role):\n",
    "    if role == 'student':\n",
    "        birthday = fake.date_of_birth(minimum_age=18, maximum_age=30)\n",
    "    else:\n",
    "        birthday = fake.date_of_birth(minimum_age=30, maximum_age=70)\n",
    "    return [birthday.isoformat(), role]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_phone_number():\n",
    "    num = fake.phone_number()\n",
    "    num = num.replace(' ','').replace('-','').replace('+7','8').replace('(','').replace(')','')\n",
    "    return num"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gen_salt_hash():\n",
    "    password = fake.password()\n",
    "    salt = fake.password(length=8)\n",
    "    return [salt, hashlib.sha1((password + salt).encode('utf-8')).hexdigest()]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "class Generator:\n",
    "\n",
    "    def __init__(self, folder):\n",
    "        self.folder = folder\n",
    "        self.student_id = 0\n",
    "        self.teacher_id = 0\n",
    "        self.account_id = 0\n",
    "        self.course_id = 0\n",
    "\n",
    "        with open(folder + 'account.csv', 'w') as f:\n",
    "            f.write('Account ID,First name,Last name,Mid name,Gender,Birthday,Role,Email,Phone number,Salt,Hash\\n')\n",
    "        with open(folder + 'student.csv', 'w') as f:\n",
    "            f.write('Student ID,Group,Account ID\\n')\n",
    "        with open(folder + 'teacher.csv', 'w') as f:\n",
    "            f.write('Teacher ID,Position,Account ID\\n')\n",
    "        with open(folder + 'course.csv', 'w') as f:\n",
    "            f.write('Course ID,Name,Teacher ID\\n')\n",
    "        with open(folder + 'lesson.csv', 'w') as f:\n",
    "            f.write('Day,Time,Place,Type,Course ID\\n')\n",
    "        with open(folder + 'student_course.csv', 'w') as f:\n",
    "            f.write('Student ID,Course ID,Score\\n')\n",
    "\n",
    "    def gen_student(self, n):\n",
    "        with open(self.folder + 'student.csv', 'a') as f:\n",
    "            for i in range(n):\n",
    "                self.student_id += 1\n",
    "                self.account_id += 1\n",
    "                f.write(','.join((str(self.student_id), gen_group(), str(self.account_id))))\n",
    "                f.write('\\n')\n",
    "                self.gen_account()\n",
    "                \n",
    "    def gen_teacher(self, n):\n",
    "        with open(self.folder + 'teacher.csv', 'a') as f:\n",
    "            for i in range(n):\n",
    "                self.teacher_id += 1\n",
    "                self.account_id += 1\n",
    "                f.write(','.join((str(self.teacher_id), gen_teacher_position(), str(self.account_id))))\n",
    "                f.write('\\n')\n",
    "                self.gen_account('teacher')\n",
    "\n",
    "    def gen_account(self, role='student'):\n",
    "        with open(self.folder + 'account.csv', 'a') as f:\n",
    "            account = [str(self.account_id)]\n",
    "            account += gen_name_gender()\n",
    "            account += gen_birthday_role(role)\n",
    "            account.append(fake.email())\n",
    "            account.append(gen_phone_number())\n",
    "            account += gen_salt_hash()\n",
    "            f.write(','.join(account))\n",
    "            f.write('\\n')\n",
    "            \n",
    "    def gen_course(self, n, n_course):\n",
    "        with open(self.folder + 'course.csv', 'a') as f:\n",
    "            for i in range(n):\n",
    "                self.course_id += 1\n",
    "                f.write(','.join((str(self.course_id), gen_course_name(n_course), str(random.randint(1, self.teacher_id)))))\n",
    "                f.write('\\n')\n",
    "                \n",
    "    def gen_lesson(self, n):\n",
    "        days = ('Пн','Вт','Ср','Чт','Пт','Сб','Вс')\n",
    "        with open(self.folder + 'lesson.csv', 'a') as f:\n",
    "            for i in range(n):\n",
    "                f.write('{},{},{}{:02}{:02},{},{}\\n'.format(random.choice(days), random.randint(1, 10),\n",
    "                                                 random.randint(1, 30), random.randint(1, 20), random.randint(1, 90),\n",
    "                                                 random.choice(('сем','лек','лаб')),\n",
    "                                                   random.randint(1, self.course_id)))\n",
    "\n",
    "    def gen_student_course(self, n):\n",
    "        with open(self.folder + 'student_course.csv', 'a') as f:\n",
    "            for i in range(n):\n",
    "                f.write('{},{},{}\\n'.format(random.randint(1, self.student_id), random.randint(1, self.course_id), random.randint(1, 100)))\n",
    "                \n",
    "\n",
    "    def gen_all(self, n_teacher, n_student):\n",
    "        self.gen_teacher(n_teacher)\n",
    "        self.gen_student(n_student)\n",
    "        self.gen_course(self.teacher_id*2, max(1, self.teacher_id/100))\n",
    "        self.gen_lesson(self.teacher_id*4)\n",
    "        self.gen_student_course(self.student_id*4)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [],
   "source": [
    "gen = Generator('csv/')\n",
    "gen.gen_all(1000, 20000)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [],
   "source": [
    "with open('csv/lesson.csv', 'w') as f:\n",
    "            f.write('Day,Time,Place,Type,Course ID\\n')\n",
    "gen.gen_lesson(4000)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined